In [1]:
from transformers import pipeline
import os
import pandas as pd
import time

In [2]:
classifier_bert = pipeline('sentiment-analysis')
classifier_finbert = pipeline('sentiment-analysis', model="ProsusAI/finbert")
classifier_roberta = pipeline('sentiment-analysis', model="cardiffnlp/twitter-roberta-base-sentiment")
classifier_roberta2 = pipeline('sentiment-analysis', model="textattack/roberta-base-SST-2")
classifier_roberta_imdb = pipeline('sentiment-analysis', model="textattack/roberta-base-imdb")
classifier_roberta_imdb2 = pipeline('sentiment-analysis', model="aychang/roberta-base-imdb")
classifier_xlnet = pipeline('sentiment-analysis', model="edwardgowsmith/xlnet-base-cased-best")
classifier_xlnet2 = pipeline('sentiment-analysis', model="textattack/xlnet-base-cased-SST-2")
classifier_electra = pipeline('sentiment-analysis', model="howey/electra-large-sst2")

#classifier2 = pipeline('distilbert-base-uncased-finetuned-sst-2-english')

#classifier("things are going pretty well")

us_fed_chairs = ["Jerome H Powell", "Janet L Yellen", "Ben S Bernanke", "Alan Greenspan"]

#NEED TO GO THROUGH LIST AND SEE WHO'S ON BOARD OF GOVORNORS
us_board_speakers = ["Alice M Rivlin", "Daniel K Tarullo", "Edward M Gramlich", 
                         "Edward W Kelley Jr", "Elizabeth A Duke", "Frederic S Mishkin",
                        "Jeremy C Stein", "Kevin M Warsh", "Lael Brainard",
                        "Laurence H Meyer", "Mark W Olson", "Randall S Kroszner",
                        "Robert W Ferguson Jr", "Sarah Bloom Raskin", "Susan Schmidt Bies"]
us_president = ["Thomas M Hoenig", "Timothy F Geithner", "William C Dudley", "William J McDonough", "Brian P Sack",
               "Charles I Plosser", "Ernest T Patrikis", "James McAndrews", "Joseph S Tracy", "Narayana Kocherlakota", 
                "Simon M Potter" ]
us_speekers_to_keep = us_fed_chairs + us_board_speakers + us_president

df = pd.read_csv("source/speech_index_manual.csv")
df = df.loc[df["author"].isin(us_speekers_to_keep)]
df.shape

Some weights of the model checkpoint at textattack/roberta-base-SST-2 were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at textattack/roberta-base-imdb were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model t

(1514, 7)

In [5]:
classifier_roberta_imdb2("the stock market is rallying")

[{'label': 'pos', 'score': 0.985923707485199}]

In [7]:
max_word_intro_length = 315
t_init = time.time()
count = 0

def del_content(i, i_init, text):
    # recursively returns list of all indexes in the speech file that should be deleted.
    # tokens to be deleted are included in the two lsits below
    del_words = ["bis", "review"] + [str(i) for i in range(0,10)]
    del_end = ["/2000"]
    list_to_delete = [] #list of indexes to be returned and deleted
    if text[i].lower() in del_words:  #if word matches words in the delete list
        list_to_delete = list_to_delete + [i]
    elif text[i][-5:] in del_end:  #if end of word matches end of words in the delete list
        list_to_delete  = list_to_delete = [i]
    else:  # not a word we want to delete, return empty list
        return []  
    if i <= i_init:  # continue checking leftward
        list_to_delete = list_to_delete + del_content(i - 1, i_init, text)
    if i >= i_init: #continue checking rightward
        try:  # TODO - try/except for the special case of the deleted item being the last item in the text
            list_to_delete = list_to_delete + del_content(i + 1, i_init, text)
        except:
            pass
    return list_to_delete

def filter_content(text):
    star_counter = 0
    for i, letter in enumerate(text):
        if letter == "*":
            star_counter += 1
        if star_counter >= 3:
            text = text[i + 1:len(text)]
            break
    split_text = text.split()
    list_to_delete = []
    for i, word in enumerate(split_text):
        if word.lower() == "bis":
            list_to_delete = list_to_delete + del_content(i, i, split_text)
    # have to be careful -- deleting items of list I'm iterating over
    # delete indexes in descending order to avoid index out of range
    list_to_delete.sort()
    for item_to_delete in reversed(list_to_delete):
        split_text.pop(item_to_delete)
        
    # TEMP TODO - only taking 300 first words because of tokenization limit
    split_text = split_text[0:max_word_intro_length]
    return " ".join(split_text)

def sentiment_analysis(speech_file):
    with open("source/txt/" + speech_file, "r", encoding="utf8") as content:
        filtered_content = filter_content(content.read())
        bert_sent = classifier_bert(filtered_content)
        finbert_sent = classifier_finbert(filtered_content)
        robertat_sent = classifier_roberta(filtered_content)
        robertat_sent2 = classifier_roberta2(filtered_content)
        robertat_imdb_sent = classifier_roberta_imdb(filtered_content)
        robertat_imdb_sent2 = classifier_roberta_imdb2(filtered_content)
        xlnet_sent = classifier_xlnet(filtered_content)
        xlnet_sent2 = classifier_xlnet2(filtered_content)
        electra_sent = classifier_electra(filtered_content)
        df.loc[df["speech"] == speech_file.split(".")[0], ["bert_sent"]] = 2 if bert_sent[0]["label"] == "POSITIVE" else 0
        df.loc[df["speech"] == speech_file.split(".")[0], ["bert_score"]] = bert_sent[0]["score"]
        if robertat_sent[0]["label"] == "LABEL_2":
            df.loc[df["speech"] == speech_file.split(".")[0], ["roberta_sent"]] = 2
        elif robertat_sent[0]["label"] == "LABEL_1":
            df.loc[df["speech"] == speech_file.split(".")[0], ["roberta_sent"]] = 1
        else:
            df.loc[df["speech"] == speech_file.split(".")[0], ["roberta_sent"]] = 0
        df.loc[df["speech"] == speech_file.split(".")[0], ["roberta_score"]] = robertat_sent[0]["score"]

        if finbert_sent[0]["label"] == "positive":
            df.loc[df["speech"] == speech_file.split(".")[0], ["finbert_sent"]] = 2
        elif finbert_sent[0]["label"] == "neutral":
            df.loc[df["speech"] == speech_file.split(".")[0], ["finbert_sent"]] = 1
        else:
            df.loc[df["speech"] == speech_file.split(".")[0], ["finbert_sent"]] = 0
        df.loc[df["speech"] == speech_file.split(".")[0], ["finbert_score"]] = finbert_sent[0]["score"]

        df.loc[df["speech"] == speech_file.split(".")[0], ["roberta_sent2"]] = 2 if robertat_sent2[0]["label"] == "LABEL_1" else 0
        df.loc[df["speech"] == speech_file.split(".")[0], ["roberta_score2"]] = robertat_sent2[0]["score"]

        df.loc[df["speech"] == speech_file.split(".")[0], ["roberta_imdb_sent"]] = 2 if robertat_imdb_sent[0]["label"] == "LABEL_1" else 0
        df.loc[df["speech"] == speech_file.split(".")[0], ["roberta_imdb_score"]] = robertat_imdb_sent[0]["score"]

        df.loc[df["speech"] == speech_file.split(".")[0], ["roberta_imdb_sent2"]] = 2 if robertat_imdb_sent2[0]["label"] == "pos" else 0
        df.loc[df["speech"] == speech_file.split(".")[0], ["roberta_imdb_score2"]] = robertat_imdb_sent2[0]["score"]

        df.loc[df["speech"] == speech_file.split(".")[0], ["xlnet_sent"]] = 2 if xlnet_sent[0]["label"] == "LABEL_1" else 0
        df.loc[df["speech"] == speech_file.split(".")[0], ["xlnet_score"]] = xlnet_sent[0]["score"]

        df.loc[df["speech"] == speech_file.split(".")[0], ["xlnet_sent2"]] = 2 if xlnet_sent2[0]["label"] == "LABEL_1" else 0
        df.loc[df["speech"] == speech_file.split(".")[0], ["xlnet_score2"]] = xlnet_sent2[0]["score"]

        df.loc[df["speech"] == speech_file.split(".")[0], ["electra_sent"]] = 2 if electra_sent[0]["label"] == "LABEL_1" else 0
        df.loc[df["speech"] == speech_file.split(".")[0], ["electra_score"]] = electra_sent[0]["score"]

for root, dirst, files in os.walk("source/txt"):
    for i, speech_file in enumerate(files):
        if speech_file.split(".")[0] in df.speech.values:
            count += 1
            if count % 25 == 0:
                print("count: " + str(count) + " - time: " + str(time.time() - t_init))
            sentiment_analysis(speech_file)
            
            
                
df.to_csv("data_files_generated/genered_speech_sentiment.csv", index=False)

count: 25 - time: 148.42158007621765
count: 50 - time: 302.3097062110901
count: 75 - time: 464.20181488990784
count: 100 - time: 629.3492469787598
count: 125 - time: 797.3333256244659
count: 150 - time: 963.4335036277771
count: 175 - time: 1128.373280286789
count: 200 - time: 1293.4189484119415
count: 225 - time: 1457.5562164783478
count: 250 - time: 1618.2392053604126
count: 275 - time: 1783.7602055072784
count: 300 - time: 1948.0580337047577
count: 325 - time: 2111.132005929947
count: 350 - time: 2274.2159287929535
count: 375 - time: 2436.1715483665466
count: 400 - time: 2599.588071346283
count: 425 - time: 2763.3741245269775
count: 450 - time: 2927.6836836338043
count: 475 - time: 3087.173216819763
count: 500 - time: 3255.5394456386566
count: 525 - time: 3421.4635877609253
count: 550 - time: 3587.8134779930115
count: 575 - time: 3753.1939866542816
count: 600 - time: 3924.6268513202667
count: 625 - time: 4091.0657374858856
count: 650 - time: 4257.20087480545
count: 675 - time: 4423.4

In [5]:
classifier_roberta = pipeline('sentiment-analysis', model="textattack/xlnet-base-cased-SST-2")
print(classifier_roberta("the chair is really bad"))



[{'label': 'LABEL_0', 'score': 0.9992764592170715}]


In [3]:
df_news = pd.read_csv("source/usnews.csv")
df_news.head()


,Unnamed: 0,id,date,texts,wsj,wapo,economy,noneconomy
0,1,830981846,1995-01-02,"In 1994, the good times for Washington area st...",0,1,1,0
1,2,842617067,1995-01-05,NEW YORK -- Small stocks rose in light trading...,1,0,0,1
2,3,830982165,1995-01-05,The Dow Jones industrial average climbed 19.17...,0,1,0,1
3,4,830982389,1995-01-08,First Months Performance Has Predicted the Mar...,0,1,0,1
4,5,842615996,1995-01-09,"PITTSBURGH -- PNC Bank Corp., moving to reduce...",1,0,0,1


In [4]:
print(df_news.loc[1])

Unnamed: 0                                                    2
id                                                    842617067
date                                                 1995-01-05
texts         NEW YORK -- Small stocks rose in light trading...
wsj                                                           1
wapo                                                          0
economy                                                       0
noneconomy                                                    1
Name: 1, dtype: object


In [5]:
def filter_content_news(text):
    split_text = text.split()
    split_text = split_text[0:300]
    return " ".join(split_text)

t_init = time.time()
count = 0
for i, row in df_news.iterrows():
    count += 1
    if count % 100 == 0:
        print("count: " + str(count) + " - time: " + str(time.time() - t_init))
    #print(row["texts"])
    filtered_content = filter_content_news(row["texts"])
    bert_sent = classifier_bert(filtered_content)
    robertat_sent = classifier_roberta(filtered_content)
    xlnet_sent = classifier_xlnet(filtered_content)
    electra_sent = classifier_electra(filtered_content)
    df_news.loc[i, ["news_bert_sent"]] = 2 if bert_sent[0]["label"] == "POSITIVE" else 0
    df_news.loc[i, ["news_bert_score"]] = bert_sent[0]["score"]
    if robertat_sent[0]["label"] == "LABEL_2":
        df_news.loc[i, ["news_roberta_sent"]] = 2
    elif robertat_sent[0]["label"] == "LABEL_1":
        df_news.loc[i, ["news_roberta_sent"]] = 1
    else:
        df_news.loc[i, ["news_roberta_sent"]] = 0
    df_news.loc[i, ["news_roberta_score"]] = robertat_sent[0]["score"]
    df_news.loc[i, ["news_xlnet_sent"]] = 2 if xlnet_sent[0]["label"] == "LABEL_1" else 0
    df_news.loc[i, ["news_xlnet_score"]] = xlnet_sent[0]["score"]
    df_news.loc[i, ["news_electra_sent"]] = 2 if electra_sent[0]["label"] == "LABEL_1" else 0
    df_news.loc[i, ["news_electra_score"]] = electra_sent[0]["score"]
    
df_news.to_csv("data_files_generated/generated_news_sentiment_1.csv", index=False)

count: 100 - time: 232.0208477973938
count: 200 - time: 486.1933515071869
count: 300 - time: 753.809323310852
count: 400 - time: 1011.5076115131378
count: 500 - time: 1267.301696062088
count: 600 - time: 1527.7114498615265
count: 700 - time: 1804.4417219161987
count: 800 - time: 2065.3353679180145
count: 900 - time: 2311.9009544849396
count: 1000 - time: 2568.752236843109
count: 1100 - time: 2831.295602083206
count: 1200 - time: 3090.180648326874
count: 1300 - time: 3348.5072615146637
count: 1400 - time: 3601.9220225811005
count: 1500 - time: 3884.8220834732056
count: 1600 - time: 4164.630816459656
count: 1700 - time: 4438.970586776733
count: 1800 - time: 4697.02149105072
count: 1900 - time: 4958.307464838028
count: 2000 - time: 5213.5413019657135
count: 2100 - time: 5471.030338525772
count: 2200 - time: 5725.711274385452
count: 2300 - time: 6007.271956682205
count: 2400 - time: 6271.266347885132
count: 2500 - time: 6525.029984235764
count: 2600 - time: 6789.878098249435
count: 2700 - 

In [116]:
df_news_generated = pd.DataFrame({'date':pd.date_range('01/01/1995', periods=10000)})
df_news_generated.head()

,date
0,1995-01-01
1,1995-01-02
2,1995-01-03
3,1995-01-04
4,1995-01-05


In [117]:

def get_prob(sent, score):
    return 1

#print(df_news["date"])
sent_list = ["news_bert_sent", "news_roberta_sent", "news_xlnet_sent", "news_electra_sent"]
score_list = ["news_bert_score", "news_roberta_score", "news_xlnet_score", "news_electra_score"]
weighted_score_list = [0 for i in range(len(sent_list))]
for i, row in df_news_generated.iterrows():
    #print(row["date"].date())
    #if str(row["date"].date()) in df_news.date.values:
    if str(row["date"].date()) in df_news["date"].values:
        #print(df_news[df_news["date"] == str(row["date"].date())].news_bert_score.values)
        articles = df_news[df_news["date"] == str(row["date"].date())]
        count_article = len(articles)
        wsj = 0
        wapo = 0
        econ = 0
        nonecon = 0
        bert_positive = 0
        bert_negative = 0
        xlnet_positive = 0
        xlnet_negative = 0
        electra_positive = 0
        electra_negative = 0
        for j in range(count_article):
            bert_positive += articles.news_bert_sent.values[j] / 2
            bert_negative += 1 - articles.news_bert_sent.values[j] / 2
            xlnet_positive += articles.news_xlnet_sent.values[j] / 2
            xlnet_negative += 1 - articles.news_xlnet_sent.values[j] / 2
            electra_positive += articles.news_bert_sent.values[j] / 2
            electra_negative += 1 - articles.news_electra_sent.values[j] / 2
            wsj += articles.wsj.values[j]
            wapo += articles.wapo.values[j]
            econ += articles.economy.values[j]
            nonecon += articles.noneconomy.values[j]
        
        df_news_generated.loc[i, "bert_positive"] = bert_positive
        df_news_generated.loc[i, "bert_negative"] = bert_negative
        df_news_generated.loc[i, "xlnet_positive"] = xlnet_positive
        df_news_generated.loc[i, "xlnet_negative"] = xlnet_negative
        df_news_generated.loc[i, "electra_positive"] = electra_positive
        df_news_generated.loc[i, "electra_negative"] = electra_negative
        
        df_news_generated.loc[i, "wsj"] = wsj
        df_news_generated.loc[i, "wapo"] = wapo
        df_news_generated.loc[i, "econ"] = econ
        df_news_generated.loc[i, "nonecon"] = nonecon

df_news_generated.to_csv("data_files_generated/generated_news_sentiment_2.csv", index=False)


In [57]:
df_news.head()

,Unnamed: 0,id,date,texts,wsj,wapo,economy,noneconomy,news_bert_sent,news_bert_score,news_roberta_sent,news_roberta_score,news_xlnet_sent,news_xlnet_score,news_electra_sent,news_electra_score
0,1,830981846,1995-01-02,"In 1994, the good times for Washington area st...",0,1,1,0,0.0,0.999352,0.0,0.460703,2.0,0.860716,0.0,0.998597
1,2,842617067,1995-01-05,NEW YORK -- Small stocks rose in light trading...,1,0,0,1,2.0,0.971528,2.0,0.705490,2.0,0.947428,0.0,0.834266
2,3,830982165,1995-01-05,The Dow Jones industrial average climbed 19.17...,0,1,0,1,0.0,0.995867,1.0,0.702038,2.0,0.803605,2.0,0.999111
3,4,830982389,1995-01-08,First Months Performance Has Predicted the Mar...,0,1,0,1,0.0,0.521700,2.0,0.702954,2.0,0.825656,2.0,0.999346
4,5,842615996,1995-01-09,"PITTSBURGH -- PNC Bank Corp., moving to reduce...",1,0,0,1,0.0,0.998336,1.0,0.573411,2.0,0.982624,0.0,0.996758
